# Miscellaneous

In [1]:
# | default_exp misc

In [2]:
# | export
import time
from torch_snippets.logger import Debug, Excep, debug_mode
from torch_snippets.markup2 import AD
from functools import wraps
from fastcore.basics import ifnone

[08/11/24 19:04:17] WARNING  Unable to load torch and dependent libraries from torch-snippets.                                                        ]8;id=984597;file:///Users/yeshwanth/Code/Personal/torch_snippets/torch_snippets/loader.py:108\loader.py]8;;\:]8;id=81772;file:///Users/yeshwanth/Code/Personal/torch_snippets/torch_snippets/loader.py:108#<module>:108\<module>:108]8;;\
                             Functionalities might be limited. pip install lovely-tensors in case there are torch related errors                                            

In [3]:
# | export
# | hide


class Timer:
    def __init__(self, N, smooth=True, mode=1):
        "print elapsed time every iteration and print out remaining time"
        "assumes this timer is called exactly N times or less"
        self.tok = self.start = time.time()
        self.N = N
        self.ix = 0
        self.smooth = smooth
        self.mode = mode
        # 0 = instant-speed, i.e., time remaining is a funciton of only the last iteration
        # Useful when you know that each loop takes unequal time (increasing/decreasing speeds)
        # 1 = average, i.e., time remaining is a function of average of all iterations
        # Usefule when you know on average each loop or a group of loops take around the same time

    def __call__(self, ix=None, info=None):
        ix = self.ix if ix is None else ix
        info = "" if info is None else f"{info}\t"
        tik = time.time()
        elapsed = tik - self.start

        if self.mode == 0:
            ielapsed = tik - self.tok
            ispeed = ielapsed
            iremaining = (self.N - (ix + 1)) * ispeed

            iunit = "s/iter"
            if ispeed < 1:
                ispeed = 1 / ispeed
                iunit = "iters/s"
            iestimate = iremaining + elapsed
            _info = f"{info}{ix+1}/{self.N} ({elapsed:.2f}s - {iremaining:.2f}s remaining - {ispeed:.2f} {iunit})"

        else:
            speed = elapsed / (ix + 1)
            remaining = (self.N - (ix + 1)) * speed
            unit = "s/iter"
            if speed < 1:
                speed = 1 / speed
                unit = "iters/s"
            estimate = remaining + elapsed
            # print(f'N={self.N} e={elapsed:.2f} _rem={self.N-(ix+1)} r={remaining:.2f} s={speed:.2f}\t\t')
            _info = f"{info}{ix+1}/{self.N} ({elapsed:.2f}s - {remaining:.2f}s remaining - {speed:.2f} {unit})"

        print(
            _info + " " * 10,
            end="\r",
        )
        self.ix += 1
        self.tok = tik


def track2(iterable, *, total=None):
    try:
        total = ifnone(total, len(iterable))
    except:
        ...
    timer = Timer(total)
    for item in iterable:
        info = yield item
        timer(info=info)
        if info is not None:
            yield  # Just to ensure the send operation stops

In [4]:
def track2(iterable, *, total=None, timer_mode=1):
    try:
        total = ifnone(total, len(iterable))
    except:
        ...
    timer = Timer(total, mode=timer_mode)
    for item in iterable:
        info = yield item
        timer(info=info)
        if info is not None:
            yield  # Just to ensure the send operation stops


l = list(range(10, 0, -1))
fact = 10
t = sum(l) / fact
for i in track2(l):
    time.sleep(i / fact)
    print()


1/10 (1.01s - 9.05s remaining - 1.01 s/iter)          
2/10 (1.91s - 7.64s remaining - 1.05 iters/s)          
3/10 (2.72s - 6.34s remaining - 1.10 iters/s)          
4/10 (3.42s - 5.13s remaining - 1.17 iters/s)          
5/10 (4.02s - 4.02s remaining - 1.24 iters/s)          
6/10 (4.53s - 3.02s remaining - 1.33 iters/s)          
7/10 (4.93s - 2.11s remaining - 1.42 iters/s)          
8/10 (5.24s - 1.31s remaining - 1.53 iters/s)          
9/10 (5.44s - 0.60s remaining - 1.66 iters/s)          


Use timer as a standalone class so you have full control on when to call a lap (most useful in while loops)...

In [5]:
N = 100
t = Timer(N)
info = None

for i in range(N):
    time.sleep(0.1)
    t(info=info)  # Lap and present the time
    if i == 50:
        print()
        info = f"My Info: {i*3.122}"

51/100 (5.29s - 5.08s remaining - 9.64 iters/s)          


... or use track2 to directly track a loop

In [6]:
N = 100
info = None

for i in (tracker := track2(range(N), total=N)):
    time.sleep(0.1)
    info = f"My Info: {i*3.122:.2f}"
    if i == N // 2:
        print()
    if i >= N // 2:
        tracker.send(info)

50/100 (5.21s - 5.21s remaining - 9.60 iters/s)          


## Warning! NEVER RUN `tracker.send(None)` as this will skip variables silently

In [7]:
# | export
# | hide


def summarize_input(args, kwargs, outputs=None):
    o = AD(args, kwargs)
    if outputs is not None:
        o.outputs = outputs
    return o.summary()


def timeit(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        Debug(f"{time.time() - s:.2f} seconds to execute `{func.__name__}`")
        return o

    return inner


def io(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        info = f"""
{time.time() - s:.2f} seconds to execute `{func.__name__}`
{summarize_input(args=args, kwargs=kwargs, outputs=o)}
        """
        Debug(info, depth=1)
        return o

    return inner

In [8]:
@io
@timeit
def foo(a, b=None):
    if b is None:
        return a + 1
    else:
        time.sleep(2)
        return a + b


with debug_mode():
    foo(10)
    foo(10, b=20)

[08/11/24 19:04:44] DEBUG    0.00 seconds to execute `foo`                                                                                            ]8;id=768985;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/1296653500.py:16\1296653500.py]8;;\:]8;id=794464;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/1296653500.py:16#inner:16\inner:16]8;;\

                    DEBUG                                                                                                                          ]8;id=196942;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/2064456364.py:12\2064456364.py]8;;\:]8;id=537244;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/2064456364.py:12#<module>:12\<module>:12]8;;\
                             0.00 seconds to execute `inner`                                                                                                                
                             args()                                                                                                                                         
                               0 - 10 (🏷️ int)                                                                                                                               
                             kwargs                                                                                                                                         
                             outputs - 11 (🏷️ int)                                                                                                                           
                                                                                                                                                                            
                                                                                                                                                                            

[08/11/24 19:04:46] DEBUG    2.00 seconds to execute `foo`                                                                                            ]8;id=200602;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/1296653500.py:16\1296653500.py]8;;\:]8;id=221957;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/1296653500.py:16#inner:16\inner:16]8;;\

                    DEBUG                                                                                                                          ]8;id=50473;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/2064456364.py:13\2064456364.py]8;;\:]8;id=285800;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/2064456364.py:13#<module>:13\<module>:13]8;;\
                             2.01 seconds to execute `inner`                                                                                                                
                             args()                                                                                                                                         
                               0 - 10 (🏷️ int)                                                                                                                               
                             kwargs                                                                                                                                         
                               b - 20 (🏷️ int)                                                                                                                               
                             outputs - 30 (🏷️ int)                                                                                                                           
                                                                                                                                                                            
                                                                                                                                                                            

In [30]:
# | export
# | hide


def tryy(func=None, *, output_to_return_on_fail=None, print_traceback=False):
    def decorator(f):
        def inner(*args, **kwargs):
            try:
                return f(*args, **kwargs)
            except Exception as e:
                if not print_traceback:
                    tb = str(e)
                else:
                    import traceback

                    tb = traceback.format_exc()
                Excep(
                    f"Error for `{f.__name__}` with \n{summarize_input(args, kwargs)}\n{tb}"
                )
                return output_to_return_on_fail

        return inner

    if callable(func):
        return decorator(func)
    return decorator

In [31]:
@tryy
def do(a, b, c):
    return 1 / 0


x = do(1, 2, c=10)
assert x is None  # tryy returns None by default

[08/11/24 19:11:33] ERROR    Error for `do` with                                                                                                      ]8;id=675640;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/3146474900.py:17\3146474900.py]8;;\:]8;id=769587;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/3146474900.py:17#inner:17\inner:17]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             division by zero                                                                                                                               

In [33]:
@tryy(output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

[08/11/24 19:11:55] ERROR    Error for `do` with                                                                                                      ]8;id=807965;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/3146474900.py:17\3146474900.py]8;;\:]8;id=923417;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/3146474900.py:17#inner:17\inner:17]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             division by zero                                                                                                                               

'😔'

Optionally print the full stacktrace if needed

In [34]:
@tryy(print_traceback=True, output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

[08/11/24 19:11:56] ERROR    Error for `do` with                                                                                                      ]8;id=118525;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/3146474900.py:17\3146474900.py]8;;\:]8;id=56484;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/3146474900.py:17#inner:17\inner:17]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             Traceback (most recent call last):                                                                                                             
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/3146474900.py", line 9, in inner                                      
                                 return f(*args, **kwargs)                                                                                                                  
                                        ^^^^^^^^^^^^^^^^^^                                                                                                                  
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_21562/2870377084.py", line 3, in do                                         
                                 return 1 / 0                                                                                                                               
                                        ~~^~~                                                                                                                               
                             ZeroDivisionError: division by zero                                                                                                            
                                                                                                                                                                            

'😔'

In [4]:
def deco(decorator):
    @wraps(decorator)
    def wrapper(*args, **kwargs):
        def real_decorator(func):
            @wraps(func)
            def inner_wrapper(*fargs, **fkwargs):
                return decorator(func, *fargs, **fkwargs)

            return inner_wrapper

        if len(args) == 1 and callable(args[0]) and not kwargs:
            # Case when B is used without arguments
            return real_decorator(args[0])
        else:
            # Case when B is used with arguments
            def custom_decorator(func):
                return decorator(func, **kwargs)

            return custom_decorator

    return wrapper

In [17]:
@deco
def B(func, *args, deco_param="default", **kwargs):
    print("B", deco_param, print("args", *args, "kwargs", **kwargs))
    return func(*args, **kwargs)

In [19]:
@B
def C(a, b, c):
    print("C")
    return a + (b * c)


C(1, 2, 3)

args 1 2 3 kwargs
B default None
C


7

In [ ]:
@B(deco_param="new_param")
def C(a, b, c):
    return a + (b * c)


# Testing
print(C(1, 2, 3))  # Outputs 'new_param' then the result 7